In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns


In [ ]:
data_path = r'C:\Users\aaani\OneDrive - Birmingham City University\Postgrad\Dissertation\Data' #path to the data directory

In [ ]:
# Define the columns to keep, this is done before loading the data to save on space complexity 

admissions_cols = ['subject_id', 'hadm_id', 'admittime', 'dischtime', 'race', 'hospital_expire_flag']
diagnoses_cols = ['subject_id', 'hadm_id', 'icd_code', 'icd_version']
d_icd_diagnoses_cols = ['icd_code', 'icd_version']
d_labitems_cols = ['itemid', 'label']
labevents_cols = ['subject_id', 'hadm_id', 'itemid', 'charttime', 'valuenum']

icustays_cols = ['subject_id', 'hadm_id', 'stay_id', 'outtime', 'los']
chartevents_cols = ['subject_id', 'hadm_id', 'itemid', 'charttime', 'valuenum', 'stay_id']
d_items_cols = ['itemid', 'label']



In [ ]:
# Initialising dfs

#hospital
admissions = pd.read_csv(f'{data_path}/admissions.csv.gz', usecols=admissions_cols)
patients = pd.read_csv(f'{data_path}/patients.csv.gz')
diagnoses = pd.read_csv(f'{data_path}/diagnoses_icd.csv.gz', usecols=diagnoses_cols)
d_icd_diagnoses = pd.read_csv(f'{data_path}/d_icd_diagnoses.csv.gz', usecols=d_icd_diagnoses_cols)
d_labitems = pd.read_csv(f'{data_path}/d_labitems.csv.gz', usecols=d_labitems_cols)
lab = pd.read_csv(f'{data_path}/labevents_last24h.csv.gz', usecols=labevents_cols, parse_dates=['charttime'])


In [ ]:
#icu
icustays = pd.read_csv(f'{data_path}/icustays.csv.gz', usecols=icustays_cols, parse_dates=['outtime'])
chartevents = pd.read_csv(f'{data_path}/chartevents_last24h.csv.gz', parse_dates=['charttime'], usecols=chartevents_cols)
d_items = pd.read_csv(f'{data_path}/d_items.csv.gz', usecols=d_items_cols)

In [ ]:
# Define the chunk size
chunk_size = 10_000_000

# This ensures the header is written only once
first_chunk = True

# a loop that reads the labevents file in chunks
for chunk in pd.read_csv(f'{data_path}/labevents.csv.gz', chunksize=chunk_size, parse_dates=['charttime']):
    
    # Merge the chunk with admissions to get the 'dischtime' for each lab event
    chunk_with_dischtime = pd.merge(chunk, admissions, on=['subject_id', 'hadm_id'], how='inner') # this is so we can calculate the final 24 hr

    # Calculate the time difference in hours from the lab event to hospital discharge
    time_delta_hours = (chunk_with_dischtime['dischtime'] - chunk_with_dischtime['charttime']).dt.total_seconds() / 3600
    
    # Filter the chunk to keep only labs from the last 24 hours before discharge
    filtered_chunk = chunk_with_dischtime[(time_delta_hours >= 0) & (time_delta_hours <= 24)]
    
    # If the filtered chunk contains any data, save it to our new CSV file
    if not filtered_chunk.empty:
        if first_chunk:
            # For the first chunk, create a new file and write the header
            filtered_chunk.to_csv('labevents_last24h.csv', index=False, mode='w', header=True)
            first_chunk = False
        else:
            # For all other chunks, append to the file without the header
            filtered_chunk.to_csv('labevents_last24h.csv', index=False, mode='a', header=False)
            
    print(f"Processed a chunk of {chunk_size} rows...")

# smaller file created!


In [ ]:
# Define the chunk size
chunk_size = 10_000_000 # 10 million rows at a time

# This flag ensures we only write the header once
first_chunk = True

# a loop that reads the chartevents file in chunks
for chunk in pd.read_csv(f'{data_path}/chartevents.csv.gz', chunksize=chunk_size, parse_dates=['charttime']):
    
    # Merge the chunk with icustays to get the 'outtime' for each event
    chunk_with_outtime = pd.merge(chunk, icustays, on=['subject_id', 'hadm_id', 'stay_id'], how='inner') # this is so we can calculate the final 24 hr
    
    # Calculate the time difference in hours from the event to discharge
    time_delta_hours = (chunk_with_outtime['outtime'] - chunk_with_outtime['charttime']).dt.total_seconds() / 3600

    # Filter the chunk to keep only events from the last 24 hours before discharge
    filtered_chunk = chunk_with_outtime[(time_delta_hours >= 0) & (time_delta_hours <= 24)]
    
    # If the filtered chunk contains any data, save it to the new CSV file
    if not filtered_chunk.empty:
        if first_chunk:
            # For the first chunk, create a new .gz file and write the header
            filtered_chunk.to_csv(
                'chartevents_last24h.csv.gz', 
                index=False, 
                mode='w', 
                header=True, 
                compression='gzip'
            )
            first_chunk = False
    else:
        # For all other chunks, add to the file without the header
        filtered_chunk.to_csv(
            'chartevents_last24h.csv.gz', 
            index=False, 
            mode='a', 
            header=False, 
            compression='gzip'
        )
            
    print(f"Processed a chunk of {chunk_size} rows...")

# compressed file created!

In [ ]:
# drop values where valuenum is null
lab= lab.dropna(subset=['valuenum'])
chartevents = chartevents.dropna(subset=['valuenum'])


In [ ]:
# merged patients to multiple table links; patient to their diagnoses, lab results, and ICU stays etc

# used left joins on all so no data is lost

# merged patients with their hospital admissions
merged = pd.merge(admissions, patients, on='subject_id', how='left')

# merge icd codes with their descriptions
diag = pd.merge(diagnoses, d_icd_diagnoses, on=['icd_code', 'icd_version'], how='left')
merged = pd.merge(merged, diag, on= ['subject_id' , 'hadm_id'], how='left') #back to the main merge

# icu stay info
merged = pd.merge(merged, icustays, on=['subject_id' , 'hadm_id'], how='left')

# add labels to lab events
lab = pd.merge(lab, d_labitems, on='itemid', how='left')
chartevents = pd.merge(chartevents, d_items, on='itemid', how='left')

# filtered to top 20 to reduce size

top_20_labs = lab['label'].value_counts().nlargest(20).index #20 most common lab tests
lab_filtered = lab[lab['label'].isin(top_20_labs)] #new fd that onlt has the top 20 lab results - smaller size

lab_features = lab_filtered.groupby(['hadm_id', 'label'])['valuenum'].agg(['mean', 'min', 'max']).unstack(level='label') #group
lab_features.columns = ['_'.join(col).strip() for col in lab_features.columns.values]
lab_features = lab_features.reset_index()

top_20_charts = chartevents['label'].value_counts().nlargest(20).index # 20 most common chart events
charts_filtered = chartevents[chartevents['label'].isin(top_20_charts)]

chartevents_features = charts_filtered.groupby(['stay_id', 'label'])['valuenum'].agg(['mean', 'min', 'max']).unstack(level='label')

chartevents_features.columns = ['_'.join(col).strip() for col in chartevents_features.columns.values]
chartevents_features = chartevents_features.reset_index()

merged = pd.merge(merged, lab_features, on=['hadm_id'], how='left')
merged = pd.merge(merged, chartevents_features, on=['stay_id'], how='left')

merged.info()


In [ ]:
merged.to_csv('final_dataset.csv.gz', index=False, compression='gzip') #saving data

In [ ]:
#df_info = pd.read_csv(f'{data_path}/final_dataset.csv.gz', nrows=1)


df = pd.read_csv(f'{data_path}/final_dataset.csv.gz')

In [ ]:
# creating a dataset that removes duplicate patients, for analysis
unique = df.drop_duplicates(subset=['subject_id']).copy()

In [ ]:
sns.set_style("whitegrid")
plt.figure(figsize=(12, 7))

sns.histplot(unique['anchor_age'], bins=30, kde=True, color='skyblue')


plt.title('Distribution of Patient Age at Admission in MIMIC-IV', fontsize=16)
plt.xlabel('Age at Admission', fontsize=12)
plt.ylabel('Number of Admissions', fontsize=12)

# Saving the plot as an image file 
plt.savefig('age_distribution.png')


In [ ]:
# Displaying unique anchor years
unique_anchor_years = unique['anchor_year_group'].unique()
unique_anchor_years.sort()
print("Unique Anchor Years:", unique_anchor_years)

In [ ]:
historical_groups = ['2008 - 2010', '2011 - 2013', '2014 - 2016']
contemporary_groups = ['2017 - 2019', '2020 - 2022']

historical = unique[unique['anchor_year_group'].isin(historical_groups)]
contemporary = unique[unique['anchor_year_group'].isin(contemporary_groups)]

historical_age_distribution = historical['anchor_age'].describe()
contemporary_age_distribution = contemporary['anchor_age'].describe()

print("Historical Age Distribution:\n", historical_age_distribution)
print("\nContemporary Age Distribution:\n", contemporary_age_distribution)



In [ ]:
sns.set_style("whitegrid")

# Create a figure for the plot
plt.figure(figsize=(14, 8))

# Plot the distribution of age for the historical cohort
sns.kdeplot(historical['anchor_age'], label='Historical Cohort (2008-2016)', color='blue', fill=True, alpha=0.5)

# Plot the distribution of age for the contemporary cohort on the same axes
sns.kdeplot(contemporary['anchor_age'], label='Contemporary Cohort (2017-2022)', color='green', fill=True, alpha=0.5)

# Add titles, labels, and a legend
plt.title('Comparison of Patient Age Distribution: Historical vs. Contemporary', fontsize=16)
plt.xlabel('Age at Admission', fontsize=12)
plt.ylabel('Density', fontsize=12)
plt.legend()
plt.grid(True)
plt.tight_layout()

plt.savefig('age_distribution_comparison.png')

#This is important as it provided visual evidence of a data drift


In [ ]:
# count the unique race categories
unique_race = unique['race'].unique()
unique_race.sort()
print("Unique Race Categories:", unique_race)

In [ ]:
# Define the mapping for the race categories for better readability

race_mapping = {
    'ASIAN - ASIAN INDIAN': 'ASIAN',
    'ASIAN - CHINESE': 'ASIAN',
    'ASIAN - KOREAN': 'ASIAN',
    'ASIAN - SOUTH EAST ASIAN': 'ASIAN',
    'BLACK/AFRICAN': 'BLACK',
    'BLACK/AFRICAN AMERICAN': 'BLACK',
    'BLACK/CAPE VERDEAN': 'BLACK',
    'BLACK/CARIBBEAN ISLAND': 'BLACK',
    'HISPANIC OR LATINO': 'HISPANIC/LATINO',
    'HISPANIC/LATINO - CENTRAL AMERICAN': 'HISPANIC/LATINO',
    'HISPANIC/LATINO - COLUMBIAN': 'HISPANIC/LATINO',
    'HISPANIC/LATINO - CUBAN': 'HISPANIC/LATINO',
    'HISPANIC/LATINO - DOMINICAN': 'HISPANIC/LATINO',
    'HISPANIC/LATINO - GUATEMALAN': 'HISPANIC/LATINO',
    'HISPANIC/LATINO - HONDURAN': 'HISPANIC/LATINO',
    'HISPANIC/LATINO - MEXICAN': 'HISPANIC/LATINO',
    'HISPANIC/LATINO - PUERTO RICAN': 'HISPANIC/LATINO',
    'HISPANIC/LATINO - SALVADORAN': 'HISPANIC/LATINO',
    'WHITE - BRAZILIAN': 'WHITE',
    'WHITE - EASTERN EUROPEAN': 'WHITE',
    'WHITE - OTHER EUROPEAN': 'WHITE',
    'WHITE - RUSSIAN': 'WHITE',
    'PORTUGUESE': 'WHITE',
    'SOUTH AMERICAN': 'HISPANIC/LATINO',
    'UNABLE TO OBTAIN': 'OTHER/UNKNOWN',
    'UNKNOWN': 'OTHER/UNKNOWN',
    'PATIENT DECLINED TO ANSWER': 'OTHER/UNKNOWN',
    'OTHER': 'OTHER/UNKNOWN',
    'AMERICAN INDIAN/ALASKA NATIVE': 'OTHER/UNKNOWN',
    'NATIVE HAWAIIAN OR OTHER PACIFIC ISLANDER': 'OTHER/UNKNOWN',
    'MULTIPLE RACE/ETHNICITY': 'OTHER/UNKNOWN'
}


unique['race_grouped'] = unique['race'].replace(race_mapping)
print(unique['race_grouped'].value_counts())


In [ ]:
historical = unique[unique['anchor_year_group'].isin(historical_groups)]
contemporary = unique[unique['anchor_year_group'].isin(contemporary_groups)]

In [ ]:
# Calculate the percentage distribution for the historical cohort
historical_race_dist = historical['race_grouped'].value_counts(normalize=True).mul(100).rename('percentage').reset_index()
historical_race_dist['Cohort'] = 'Historical (2008-2016)'

# Calculate the percentage distribution for the contemporary cohort
contemporary_race_dist = contemporary['race_grouped'].value_counts(normalize=True).mul(100).rename('percentage').reset_index()
contemporary_race_dist['Cohort'] = 'Contemporary (2017-2022)'


combined_race_dist = pd.concat([historical_race_dist, contemporary_race_dist])

# Visualisation 
plt.figure(figsize=(15, 8))
sns.set_style("whitegrid")

# Create the bar plot
barplot = sns.barplot(data=combined_race_dist, x='race_grouped', y='percentage', hue='Cohort', palette='viridis')

# Add titles and labels
plt.title('Comparison of Patient Race Distribution: Historical vs. Contemporary', fontsize=16)
plt.xlabel('Race', fontsize=12)
plt.ylabel('Percentage of Patients (%)', fontsize=12)
plt.xticks(rotation=45, ha='right') # Rotate labels for better readability
plt.tight_layout()

# Save the plot
plt.savefig('race_distribution_comparison.png')

In [ ]:
# Calculate the percentage distribution for the cohorta
hist_gender_dist = historical['gender'].value_counts(normalize=True).mul(100).rename('percentage').reset_index()
hist_gender_dist['Cohort'] = 'Historical (2008-2016)'

cont_gender_dist = contemporary['gender'].value_counts(normalize=True).mul(100).rename('percentage').reset_index()
cont_gender_dist['Cohort'] = 'Contemporary (2017-2022)'

# Combine the two distributions into a single DataFrame for plotting
combined_gender_dist = pd.concat([hist_gender_dist, cont_gender_dist])


# visualisation 

plt.figure(figsize=(10, 7))
sns.set_style("whitegrid")

# Create the bar plot
barplot = sns.barplot(
    data=combined_gender_dist, 
    x='gender', 
    y='percentage', 
    hue='Cohort',
    palette='magma'
)

for p in barplot.patches:
    if p.get_height() > 0:
        barplot.annotate(f'{p.get_height():.1f}%',           # % formatted to one decimal place
                    (p.get_x() + p.get_width() / 2., p.get_height()), #  coordinates of the text
                    ha='center', va='center',          # center alignment
                    xytext=(0, 9),                     # put text slightly above the bar
                    textcoords='offset points',
                    fontsize=12)


plt.legend(bbox_to_anchor=(1.02, 1), loc='upper left', borderaxespad=0.) # Move the legend outside the plot area

# Add titles and labels
plt.title('Comparison of Patient Gender Distribution by Cohort', fontsize=16)
plt.xlabel('Gender', fontsize=12)
plt.ylabel('Percentage of Patients (%)', fontsize=12)
plt.xticks(ticks=['F', 'M'], labels=['Female', 'Male']) 


plt.savefig('Images/gender_distribution_comparison.png')

In [ ]:
historical['Cohort'] = 'Historical (2008-2016)'
contemporary['Cohort'] = 'Contemporary (2017-2022)'

# Combine the two patient-level dataframes
combined_df = pd.concat([historical, contemporary])

# Map the hospital_expire_flag to readable labels for the legend
combined_df['Outcome'] = combined_df['hospital_expire_flag'].map({0: 'Survived', 1: 'Died'})


# two subplots side-by-side to compare the cohorts
fig, axes = plt.subplots(1, 2, figsize=(18, 8), sharey=True)
sns.set_style("whitegrid")

# Plot for Historical Cohort
sns.histplot(
    data=combined_df[combined_df['Cohort'] == 'Historical (2008-2016)'],
    x='race_grouped',
    hue='Outcome',
    multiple='fill',  # stacked bar
    stat='percent',
    shrink=0.8,
    ax=axes[0],
    palette={'Survived': 'skyblue', 'Died': 'coral'}
)
axes[0].set_title('Historical Cohort (2008-2016)', fontsize=14)
axes[0].set_xlabel('Race', fontsize=12)
axes[0].set_ylabel('Percentage of Patients (%)', fontsize=12)
axes[0].tick_params(axis='x', rotation=45)

# Plot for Contemporary Cohort
sns.histplot(
    data=combined_df[combined_df['Cohort'] == 'Contemporary (2017-2022)'],
    x='race_grouped',
    hue='Outcome',
    multiple='fill',
    stat='percent',
    shrink=0.8,
    ax=axes[1],
    palette={'Survived': 'skyblue', 'Died': 'coral'}
)
axes[1].set_title('Contemporary Cohort (2017-2022)', fontsize=14)
axes[1].set_xlabel('Race', fontsize=12)
axes[1].set_ylabel('') # Hide y-axis label for the second plot for clarity
axes[1].tick_params(axis='x', rotation=45)

# Add Percentage Labels to Both Plots
for ax in axes:
    for container in ax.containers:
        # Get the labels for each bar in the container
        labels = [f'{h.get_height()*100:.1f}%' for h in container]
        ax.bar_label(container, labels=labels, label_type='center', fontsize=10, color='black', weight='bold')


# titles
fig.suptitle('In-Hospital Mortality Rate by Race and Cohort', fontsize=20)
plt.tight_layout(rect=[0, 0, 1, 0.96]) # Adjust layout to make room for suptitle

# Save the plot
plt.savefig('Images/outcome_by_race_comparison.png')

In [ ]:

df_sample = df.sample(n=1000, random_state=1)
plt.figure(figsize=(20, 10))
sns.heatmap(df_sample.isnull(), cbar=False, cmap='viridis')
plt.title('Missing Data Heatmap (Sample of 1000 Rows)')
plt.show()

In [ ]:
top_features = [
    # ICU Vitals and Stays
    'los', 
    'mean_Heart Rate', 
    'mean_Respiratory Rate', 
    'mean_Glucose', 
    'mean_Temperature Fahrenheit',
    
    # Key Lab Values
    'mean_Creatinine',
    'mean_Potassium',
    'mean_Sodium',
    'mean_White Blood Cells',
    'mean_Hemoglobin',
    'mean_Platelet Count'
]

corr_matrix = df[top_features].corr()

mask = np.triu(np.ones_like(corr_matrix, dtype=bool)) # to only show bottom half


plt.figure(figsize=(10, 8))
sns.heatmap(corr_matrix, annot=True, cmap='coolwarm', fmt=".2f", mask=mask)
plt.title('Feature Correlation Heatmap')
plt.show()

plt.savefig('Images/feature_correlation_heatmap.png')

In [ ]:
plt.figure(figsize=(10, 7))
sns.set_style("whitegrid")

# Create the boxplot 
sns.boxplot(
    data=df, 
    x='hospital_expire_flag', 
    y='los', 
    hue='hospital_expire_flag', # Assign the x-variable to hue
    palette="viridis",
    legend=False # Hides legend
)

# Change the x-axis labels
plt.xticks(ticks=[0, 1], labels=['Survived', 'Died'], fontsize=12)

# Add titles and labels
plt.title('In-Hospital Mortality is Associated with Longer Stays', fontsize=16)
plt.xlabel('Patient Outcome', fontsize=14)
plt.ylabel('Length of Stay in Days', fontsize=14)

# Zoom in on the y-axis to show box plots better
plt.ylim(0, 20)

plt.show()

plt.savefig('Images/in_hospital_mortality_longer_stays.png')